In [1]:
import phippery
from phippery.utils import *

import xarray as xr
import numpy as np
import pandas as pd

import os
import warnings
warnings.filterwarnings('ignore')

import ot

from Bio.Align import substitution_matrices

In [2]:
matrix_name = "BLOSUM62"                # substitution matrix for amino acid similarity
metric = "smooth_flank_1_enr_diff_sel"  # scaled differential selection name

# epitope region position boundaries
epitope_regions = {
    "FP"  : [ 805,  835],
    "SHH" : [1135, 1170]
}

# output directory
outdir = 'results'
if not os.path.exists(outdir): os.mkdir(outdir)

# input file
ds = phippery.load("LK_DMS_1rep_layered.phip")
ds

<xarray.Dataset>
Dimensions:                      (peptide_id: 24840, peptide_metadata: 6, sample_id: 45, sample_metadata: 31)
Coordinates:
  * peptide_id                   (peptide_id) int64 0 1 2 ... 24837 24838 24839
  * sample_id                    (sample_id) int64 1461 1421 1432 ... 1467 1420
  * sample_metadata              (sample_metadata) object 'Cohort' ... 'visit...
  * peptide_metadata             (peptide_metadata) object 'Oligo' ... 'is_wt'
Data variables:
    counts                       (peptide_id, sample_id) int64 0 0 0 ... 71 1270
    sample_table                 (sample_id, sample_metadata) object nan ... nan
    peptide_table                (peptide_id, peptide_metadata) object 'aggaa...
    cpm                          (peptide_id, sample_id) float64 0.0 ... 1.35...
    size_factors                 (peptide_id, sample_id) float64 0.0 ... 1.04...
    enrichment                   (peptide_id, sample_id) float64 0.1664 ... 8...
    neg_binom_mlxp               (peptide_id, sample_id) float64 0.0 ... 3.37
    smooth_flank_1_enr_diff_sel  (peptide_id, sample_id) float64 0.4527 ... -...
Attributes:
    collapsed_variable:  None

In [3]:
# sample IDs for LK individuals
lk_infant_sams = sample_id_coordinate_subset(ds,where='sample_group',is_equal_to='LK African Infant').tolist()
lk_mother_sams = sample_id_coordinate_subset(ds,where='sample_group',is_equal_to='LK African Mother').tolist()

In [4]:
# set up cost matrix
alphabet_list = list('ARNDCQEGHILKMFPSTWYV')
Naa = len(alphabet_list)
substitution_matrix = substitution_matrices.load(matrix_name)

nthroot=7.
maxMij = np.exp(np.max(-substitution_matrix)/nthroot)
cost_matrix=[]
for aa in alphabet_list:
    row = [-x/nthroot for x in substitution_matrix[aa,:][:Naa]]
    cost_row = (np.exp(row)).tolist() + [maxMij for i in range(Naa)]
    cost_matrix.append(cost_row)

for aa in alphabet_list:
    row = [-x/nthroot for x in substitution_matrix[aa,:][:Naa]]
    cost_row = [maxMij for i in range(Naa)] + (np.exp(row)).tolist()
    cost_matrix.append(cost_row)

In [5]:
# Retrieve "histogram" representation of escape profile at a site, normalized to unit "area"
# - first 20 bins are magnitude of escape (scaled differential selection < 0) for each amino acid
# - last 20 bins are positive scaled differential selection for each amino acid
def get_loc_escape_data(
    ds,
    sid,
    loc
):
    my_ds = ds.loc[
                dict(
                    peptide_id=peptide_id_coordinate_subset(ds,where='Loc',is_equal_to=loc),
                    sample_id=sid
                )
            ]
    
    diff_sel = my_ds[metric].to_pandas().to_numpy().flatten()
    
    my_df = my_ds.peptide_table.loc[:,['aa_sub']].to_pandas()
    my_df['diff_sel'] = diff_sel
    
    esc_data_neg=[]
    esc_data_pos=[]
    for aa in alphabet_list:
        val = my_df[my_df['aa_sub']==aa]['diff_sel'].item()
        if val>0:
            esc_data_neg.append(0)
            esc_data_pos.append(val)
        else:
            esc_data_neg.append(-val)
            esc_data_pos.append(0)
    
    esc_data = esc_data_neg + esc_data_pos
    
    if np.sum(esc_data)==0:
        return esc_data
    else:
        return esc_data/np.sum(esc_data)

In [6]:
# Compute site weight in the comparison of two escape profiles
# The weight is the minimum of the relative contribution of that site to the respective escape profile
def get_weights(
    ds,
    sid1,
    sid2,
    loc_start,
    loc_end
):   
    loc_sums1=[]
    loc_sums2=[]
    for loc in range(loc_start, loc_end+1):
        ds1 = ds.loc[
                dict(
                    peptide_id=peptide_id_coordinate_subset(ds,where='Loc',is_equal_to=loc),
                    sample_id=sid1
                    )
                ]

        diff_sel1 = ds1[metric].to_pandas().to_numpy().flatten()
        loc_sums1.append(0)
        for val in diff_sel1:
            loc_sums1[-1] = loc_sums1[-1] + abs(val)
        
        ds2 = ds.loc[
                dict(
                    peptide_id=peptide_id_coordinate_subset(ds,where='Loc',is_equal_to=loc),
                    sample_id=sid2
                    )
                ]
        
        diff_sel2 = ds2[metric].to_pandas().to_numpy().flatten()
        loc_sums2.append(0)
        for val in diff_sel2:
            loc_sums2[-1] = loc_sums2[-1] + abs(val)
    
    loc_sums1 = loc_sums1/np.sum(loc_sums1)
    loc_sums2 = loc_sums2/np.sum(loc_sums2)
        
    weights={}
    total=0
    for i,loc in zip(range(loc_end-loc_start+1), range(loc_start, loc_end+1)):
        val = min(loc_sums1[i], loc_sums2[i])
        if loc==1136 or loc==1147: val=0
        total = total+val
        weights[loc] = val
    
    weights = {k: v/total for k,v in weights.items()}

    return weights

In [7]:
# Compute epitope region similarity between two escape profiles
def region_compare(ds, sid1, sid2, loc_start, loc_end):
    weights = get_weights(ds, sid1, sid2, loc_start, loc_end)
    region_sim=0
    for loc in range(loc_start, loc_end+1):
        a    = get_loc_escape_data(ds,sid1,loc)
        b    = get_loc_escape_data(ds,sid2,loc)
        cost = ot.emd2(a, b, cost_matrix)
        sim  = weights[loc]/cost
        if (np.sum(a)==0 and np.sum(b)>0) or (np.sum(a)>0 and np.sum(b)==0):
            sim = 0
        region_sim = region_sim + sim
        
    return region_sim

In [8]:
output_df = pd.DataFrame(columns=['part_ID_1','group_1','part_ID_2','group_2','epitope_region','similarity'])

group_list = [lk_infant_sams, lk_mother_sams]
desc_list  = ['LK_Infant', 'LK_Mother']

other_group_list = copy.deepcopy(group_list)
other_desc_list  = copy.deepcopy(desc_list)
for group1, desc1 in zip(group_list, desc_list):
    for group2, desc2 in zip(other_group_list, other_desc_list):
        for region in ['FP', 'SHH']:
            print(region,desc1,desc2)
            for sid1 in group1:
                part_id1 = ds.loc[dict(sample_id=sid1)].sample_table.loc['participant_ID'].values
                for sid2 in group2:
                    if desc1==desc2 and sid1>=sid2: continue
                    part_id2 = ds.loc[dict(sample_id=sid2)].sample_table.loc['participant_ID'].values
                    sim = region_compare(ds, sid1, sid2, epitope_regions[region][0], epitope_regions[region][1])
                    output_df.loc[len(output_df.index)] = [part_id1, desc1, part_id2, desc2, region, sim]
    other_group_list.remove(group1)
    other_desc_list.remove(desc1)

output_df.to_csv(f'{outdir}/LK_escape_compare_sims.csv', index=False, na_rep="NA")
output_df

FP LK_Infant LK_Infant
SHH LK_Infant LK_Infant
FP LK_Infant LK_Mother
SHH LK_Infant LK_Mother
FP LK_Mother LK_Mother
SHH LK_Mother LK_Mother


,part_ID_1,group_1,part_ID_2,group_2,epitope_region,similarity
0,ptnum_8,LK_Infant,ptnum_142,LK_Infant,FP,1.374935
1,ptnum_8,LK_Infant,ptnum_18,LK_Infant,FP,1.371606
2,ptnum_8,LK_Infant,ptnum_41,LK_Infant,FP,1.425037
3,ptnum_50,LK_Infant,ptnum_8,LK_Infant,FP,1.434415
4,ptnum_50,LK_Infant,ptnum_133,LK_Infant,FP,1.551646
...,...,...,...,...,...,...
1975,ptnum_41,LK_Mother,ptnum_203,LK_Mother,SHH,0.927252
1976,ptnum_41,LK_Mother,ptnum_204,LK_Mother,SHH,0.985854
1977,ptnum_41,LK_Mother,ptnum_205,LK_Mother,SHH,0.899583
1978,ptnum_41,LK_Mother,ptnum_25,LK_Mother,SHH,1.092964


In [13]:
for region in ['FP','SHH']:
    print(f'Mother-Mother pairs in {region}:',
          output_df[(output_df['group_1']=='LK_Mother') & (output_df['group_2']=='LK_Mother') & (output_df['epitope_region']==region)].shape[0])

Mother-Mother pairs in FP: 595
Mother-Mother pairs in SHH: 595


In [14]:
for region in ['FP','SHH']:
    print(f'Infant-Infant pairs in {region}:',
          output_df[(output_df['group_1']=='LK_Infant') & (output_df['group_2']=='LK_Infant') & (output_df['epitope_region']==region)].shape[0])

Infant-Infant pairs in FP: 45
Infant-Infant pairs in SHH: 45


In [15]:
for region in ['FP','SHH']:
    print(f'Infant-Mother pairs in {region}:',
          output_df[(output_df['group_1']=='LK_Infant') & (output_df['group_2']=='LK_Mother') & (output_df['epitope_region']==region)].shape[0])

Infant-Mother pairs in FP: 350
Infant-Mother pairs in SHH: 350


In [18]:
for region in ['FP','SHH']:
    print(f'Matched Infant-Mother pairs in {region}:',
          output_df[(output_df['group_1']=='LK_Infant') & (output_df['group_2']=='LK_Mother') & (output_df['part_ID_1']==output_df['part_ID_2']) & (output_df['epitope_region']==region)].shape[0])

Matched Infant-Mother pairs in FP: 9
Matched Infant-Mother pairs in SHH: 9
